## Assignment 2

Created on Thu Mar  1 10:52:39 2018

@author: ktaranov

In [9]:
import nltk 
import numpy as np

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('universal_tagset')
# nltk.download('tagsets')

In [10]:
# PoS tagging is the process of tagging a word in a text
# where the tag is a particular part of speech mark

# The 12 universal tags are:

# VERB - verbs (all tenses and modes)
# NOUN - nouns (common and proper)
# PRON - pronouns 
# ADJ - adjectives
# ADV - adverbs
# ADP - adpositions (prepositions and postpositions)
# CONJ - conjunctions
# DET - determiners
# NUM - cardinal numbers
# PRT - particles or other function words
# X - other: foreign words, typos, abbreviations
# . - punctuation

# Examples of tagging:

text = nltk.word_tokenize("ETH is the best university in the world.")
nltk.pos_tag(text,tagset='universal')
print(text)
text = nltk.word_tokenize("Dogs are animals.")
nltk.pos_tag(text,tagset='universal')
print(text)

['ETH', 'is', 'the', 'best', 'university', 'in', 'the', 'world', '.']
['Dogs', 'are', 'animals', '.']


### In this exercise, you should implement your PoS tagging using a Hidden Markov model.
For learning your model the following Tagged Corpora can be used.

In [11]:
# nltk.download('brown')
# nltk.download('treebank')

#from nltk.corpus import treebank as corpus
from nltk.corpus import brown as corpus

# We add an artificial "START" and "END" tags at the beginning and at the end of each sentence

tagged_words = [ ]
all_tags = [ ]

for sent in corpus.tagged_sents(tagset='universal'): # get tagged sentences
    tagged_words.append( ("START", "START") )
    all_tags.append("START")
    for (word, tag) in sent:
        all_tags.append(tag)
        tagged_words.append( (tag, word) ) 
    tagged_words.append( ("END", "END") )
    all_tags.append("END")

### Estimating probabilities section
For Viterbi algorithm we need to compute:
* The maximum likelihood estimate of a transition probability $P(t_{i} | t_{i-1}) = C(t_{i-1}, t_{i})/C(t_{i-1})$
* Emission probabilities $P(w_{i} | t_{i}) =  C(t_{i}, w_{i}) / C(t_{i})$
 

In [12]:
# C(t_{i-1}, t_{i}):
cfd_tags= nltk.ConditionalFreqDist(nltk.bigrams(all_tags))
# Transition probabilities P(t_{i} | t_{i-1})
cpd_tags = nltk.ConditionalProbDist(cfd_tags, nltk.MLEProbDist)

# Example:
# C('DET', 'NOUN'):
print("Frequency of C('DET', 'NOUN') is ", cfd_tags['DET']['NOUN'] )
# P('NOUN' | 'DET')
print("Probability of P('NOUN' | 'DET') is ", cpd_tags['DET'].prob('NOUN') )


# C(t_{i}, w_{i})
cfd_tagwords = nltk.ConditionalFreqDist(tagged_words)
# Emission probabilities P(w_{i} | t_{i})
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords, nltk.MLEProbDist)

# Example:
# C('DET', 'NOUN'):
print("Frequency of C('DET', 'the') is ", cfd_tagwords['DET']['the'] )
# P('the' | 'DET')
print("Probability of P('the' | 'DET') is ", cpd_tagwords['DET'].prob('the') )

Frequency of C('DET', 'NOUN') is  85838
Probability of P('NOUN' | 'DET') is  0.6264678621213117
Frequency of C('DET', 'the') is  62710
Probability of P('the' | 'DET') is  0.45767375327509324


### Task 1:
Estimate the probability of the tag sequence "NOUN VERB VERB ." for the word sequence "birds can fly."  $P(t^{n} | w^{n} )$ using the transition and emission probabilities estimated above.

 

In [13]:
prob_tagsequence = cpd_tags["START"].prob("NOUN")  * cpd_tagwords["NOUN"].prob("birds") * \
                   cpd_tags["NOUN"].prob("VERB")   * cpd_tagwords["VERB"].prob("can") *\
                   cpd_tags["VERB"].prob("VERB")   * cpd_tagwords["VERB"].prob("fly") * \
                   cpd_tags["VERB"].prob(".")      * cpd_tagwords["."].prob(".") * \
                   cpd_tags["."].prob("END")

print("The probability of the tag sequence 'NOUN VERB VERB .' for 'birds can fly.' is:", prob_tagsequence)

The probability of the tag sequence 'NOUN VERB VERB .' for 'birds can fly.' is: 6.5932971745681245e-15


### Task 2:
Implement the Viterbi algorithm for PoS tagging.

In [14]:
def viterbi(sentence):
    # Step 1. initialization step
 
    distinct_tags = np.array(list(set(all_tags)))
    tagslen = len(distinct_tags)

    sentlen = len(sentence)
    viterbi = np.zeros((tagslen, sentlen+1) ,dtype=float)
    backpointer = np.zeros((tagslen, sentlen+1) ,dtype=np.uint32)
    
 
   
    # Step 1. initialization step
    for s, tag in enumerate(distinct_tags):
        viterbi[s,0] =  cpd_tags["START"].prob(tag) * cpd_tagwords[tag].prob( sentence[0] )
        backpointer[s,0] = 0
        print("Viterbi probability V( {1} ,{0} ) = {2}".format(sentence[0], tag, viterbi[s,0]))
    print(20*'=')

    
    # Step 2. recursion step
    for t in range(1, sentlen):
        for s, tag in enumerate(distinct_tags):
            current_viterbi = np.zeros( tagslen ,dtype=float)
            for sprime, predtag in enumerate(distinct_tags):
                current_viterbi[sprime] = viterbi[sprime,t-1] * \
                                          cpd_tags[predtag].prob(tag) * \
                                          cpd_tagwords[tag].prob(sentence[t])
            backpointer[s,t] = np.argmax(current_viterbi)
            viterbi[s,t] = max(current_viterbi)
            print("Viterbi probability V( {1} ,{0} ) = {2}".format(sentence[t],tag,  viterbi[s,t]))
            
        print(20*'=')
        
            
    # Step 3. termination step
    current_viterbi = np.empty( tagslen ,dtype=float)
    ind_of_end = -1
    for s, tag in enumerate(distinct_tags):
        if tag == "END":
            ind_of_end  = s
        current_viterbi[s] = viterbi[s,sentlen-1] * cpd_tags[tag].prob("END") 
           
    backpointer[ind_of_end,sentlen] = np.argmax(current_viterbi)
    viterbi[ind_of_end,sentlen] = max(current_viterbi)
            
    # Step 3. backtrace the path
    best_tagsequence = [ ]
    prob_tagsequence = 1.0
    prevind  = ind_of_end
    for t in range(sentlen,0,-1):
        if(t!=sentlen):
            prob_tagsequence=prob_tagsequence*viterbi[prevind,t]
        prevind = backpointer[prevind,t]
        best_tagsequence.append(distinct_tags[prevind])
    best_tagsequence.reverse()

    return best_tagsequence, prob_tagsequence

    

### Task 3:

Try to tag the sentences below.
- Why does it fail to predict tags for the sentence about ETH?  
- (Could it just mislabel the sentence because it disagreed with the statement?)

In [15]:
sentence =  nltk.word_tokenize("The dog is mine.")    
#sentence =  nltk.word_tokenize("Have a nice day.")
#sentence = nltk.word_tokenize("ETH is the best university in the world.")
best_tagsequence, prob_tagsequence = viterbi(sentence)



    
print("The sentence was: ")
for w in sentence: print( w )
print ()
print("The best tag sequence is:")
for t in best_tagsequence: print (t)
print()
print("The probability of the best tag sequence is:", prob_tagsequence)

Viterbi probability V( PRON ,The ) = 0.0
Viterbi probability V( . ,The ) = 0.0
Viterbi probability V( START ,The ) = 0.0
Viterbi probability V( X ,The ) = 0.0
Viterbi probability V( DET ,The ) = 0.011305478033945441
Viterbi probability V( CONJ ,The ) = 0.0
Viterbi probability V( PRT ,The ) = 0.0
Viterbi probability V( NUM ,The ) = 0.0
Viterbi probability V( END ,The ) = 0.0
Viterbi probability V( ADP ,The ) = 0.0
Viterbi probability V( ADV ,The ) = 0.0
Viterbi probability V( NOUN ,The ) = 0.0
Viterbi probability V( ADJ ,The ) = 0.0
Viterbi probability V( VERB ,The ) = 0.0
Viterbi probability V( PRON ,dog ) = 0.0
Viterbi probability V( . ,dog ) = 0.0
Viterbi probability V( START ,dog ) = 0.0
Viterbi probability V( X ,dog ) = 0.0
Viterbi probability V( DET ,dog ) = 0.0
Viterbi probability V( CONJ ,dog ) = 0.0
Viterbi probability V( PRT ,dog ) = 0.0
Viterbi probability V( NUM ,dog ) = 0.0
Viterbi probability V( END ,dog ) = 0.0
Viterbi probability V( ADP ,dog ) = 0.0
Viterbi probability V